<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Dash - Dynamic Link On Plotly Data Label Click

**Tags:** #dash #plotly #naas #analytics

**Author:** [Oguz Akif Tufekcioglu](https://www.linkedin.com/in/oguzakiftufekcioglu/)

This notebook enables you to generate plotly plots with clickable data points that sends to their urls in the dataframe.

## Input

### Import libraries

In [ ]:
import os
try:
    import dash
except:
    !pip install dash --user
    import dash
try:
    import dash_bootstrap_components as dbc
except:
    !pip install dash_bootstrap_components --user
    import dash_bootstrap_components as dbc
    
import webbrowser
from dash.dependencies import Input, Output
from dash import html, dcc
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objects as go

### Setup Variables

In [ ]:
DASH_PORT = 8050

## Model

### Initialize Dash app

In [ ]:
# app = dash.Dash(
#     requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/', 
#     external_stylesheets=[dbc.themes.BOOTSTRAP],
#     meta_tags=[{'name':'viewport', 'content':'width=device-width, initial-scale=1.0'}]
# ) 

# if you are not in Naas
app = dash.Dash() 

### Get stock prices

In [ ]:
df = px.data.stocks() #reading stock price dataset
print("Data fetched:", len(df))
df.head()

### Add an url column

In [ ]:
# add a url for each stock in the dataset into urls column
df["urls"] = "https://www.naas.ai/"

df.head()

### Create chart

In [ ]:
def stock_prices(ticker, label):
    # Function for creating line chart showing Google stock prices over time 
    fig = go.Figure(
        [
            go.Scatter(
                x=df['date'],
                y=df[ticker],
                line=dict(color='firebrick', width=4),
                name=label
            )
        ]
    )
    fig.update_layout(
        title='Prices over time',
        xaxis_title='Dates',
        yaxis_title='Prices'
    )
    return fig

fig = stock_prices(ticker='GOOG', label="Google")
fig

### Create app layout

In [ ]:
app.layout = html.Div(
    id ='parent',
    children=[
        html.H1(
            id='H1',
            children='Dynamic link on label click on plotly chart python',
            style = {
                'textAlign': 'center',
                'marginTop': 40,
                'marginBottom': 40
            }
        ),
        dcc.Graph(
            id='line-plot', 
            figure=px.line(data_frame=df, x='date', y='GOOG', title='Google stock prices over time',custom_data=("urls",))
        ),
    ]
)

### Init callback function to open url in new tab when data point is clicked

In [ ]:
@app.callback(Output('line-plot', 'children'),[Input('line-plot', 'clickData')])
def open_url(clickData):
    if clickData != 'null':
        url = clickData['points'][0]['customdata'][0]
        webbrowser.open_new_tab(url)
    else:
        raise PreventUpdate

## Output

### Generate URL and show logs

In [ ]:
if __name__ == '__main__':
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")